### 处理数据获得 224 * 224 的CT标签图像，融合后的图像，Lable图像

此代码的功能：
读入CT转为HU
读入PET转为HU，PET上采样为与CT一致
读入Lable的BMP图像

将PET和CT加权融合

将融合图像fusionImage,CT原图像，Label图像按[y1:y2,x1,x2]裁剪到大小为256 * 256

将256 * 256图像（fusionImage，CT原图像）归一化到0-255

将归一化图像和Label图像resize为224 * 224


针对yuanfazhao_05处理数据，处理的结果还是保存在yuanfazhao_05。再把处理得到的结果取出来存放到其他地方。文件命名没有变。

In [1]:
import os
import cv2
import shutil
import pydicom
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline 
%config InlineBackend.figure_format = 'svg' #矢量图比较高清

In [2]:
# 归一化，将值归一化到0~255
def lumTrans(img):
    # 将所有体素的值截取到-1200到600这个范围
    lungwin = np.array([-1200.,600.])
    newimg = (img-lungwin[0])/(lungwin[1]-lungwin[0])
    newimg[newimg<0]=0
    newimg[newimg>1]=1
    # 缩放到0~255
    newimg = (newimg*255).astype('uint8')
    return newimg

In [3]:
# 加载PET,CT图像，2D加载
def load_slice(path):
    slice = pydicom.dcmread(path)
    slice_arr = slice.pixel_array # (H,W)
    
    intercept = slice.RescaleIntercept
    slope = slice.RescaleSlope
    
    return slice_arr,intercept,slope

In [4]:
# 加载标签图像
def load_label(path):
    label = Image.open(path) # (W,H)
    label_img = np.array(label).astype(np.uint8) # (H,W)
    
    return label_img

In [5]:
# mask图的保存,label_img为（H,W）形状
def save_label(label_img):
    # 设置具有恒定阈值的转换表（因为PIL的模式“1"默认以125为阈值，不便于保存）
    threshold = 1
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    
    labelimg = Image.fromarray(label_img) # (W,H)
    img = labelimg.point(table,"1")
    
    return img

In [15]:
# HU值和SUV值融合,再归一化

def prep(ct_path,pet_path,label_path):
    
    # 读入ct
    ct_img,ct_intercept,ct_slope = load_slice(ct_path) # （H,W) 512 int16 (-2000,0,4025)
    
    # 读入pet
    pet_img,pet_intercept,pet_slope = load_slice(pet_path) # (H,W) 128 int16 (0,1000多)
    
    # 读入label
    label_img = load_label(label_path) # (H,W) 512 uint8 (0,1)
    
    # 计算HU和SUV值
    ct_hu = ct_slope * ct_img + ct_intercept  # 512
    pet_suv = pet_slope * pet_img + pet_intercept # 128
    
    # 对pet上采样，参数中的(512,512)为（W,H）Lanczos插值
    pet_up_img = cv2.resize(pet_suv,(512,512),interpolation=cv2.INTER_LANCZOS4) #(H,W) 512 int16 (可能有负数)
    
    # pet与ct图像混合,addWeighted
    alpha =0.7        
    beta  =0.3 
    gamma =0.0
    fusion_img = cv2.addWeighted(ct_hu,alpha,pet_up_img,beta,gamma) # (H,W) 512 float64
    
    # 裁剪为,numpy切片[y1:y2,x1:x2]
    fusion_crop = fusion_img[109:413,91:395] # (H,W) 256 float64
    
    # 归一化 （用于保存的）
    fusion = lumTrans(fusion_crop) # (H,W) 256 uint8 
    fusion_re = cv2.resize(fusion,(224,224),interpolation=cv2.INTER_LANCZOS4)
    
    # label裁剪 （用于保存的）
    label_crop = label_img[109:413,91:395] # (H,M) （256,256)
    label_re = cv2.resize(label_crop,(224,224),interpolation=cv2.INTER_LANCZOS4)
    
    # ct处理（用于保存的）
    ct_crop = ct_hu[109:413,91:395] # 256
    ct_lum = lumTrans(ct_crop) # 256 
    ct_re = cv2.resize(ct_lum,(224,224),interpolation=cv2.INTER_LANCZOS4)
    
    # 构建保存地址
    patient_ct,filename = os.path.split(ct_path)
    patient = os.path.split(patient_ct)[0]
    
    ct_dir = os.path.join(patient,"CTCrop")
    if not os.path.exists(ct_dir):
        os.makedirs(ct_dir)
    ct_dst = "%s%s" % (os.path.join(ct_dir,filename[:3]),".bmp")
   
    fu_dir = os.path.join(patient,"Fusion")
    if not os.path.exists(fu_dir):
        os.makedirs(fu_dir)
    fusion_dst = "%s%s" % (os.path.join(fu_dir,filename[:3]),".bmp")
    
    la_dir = os.path.join(patient,"LabelCrop")
    if not os.path.exists(la_dir):
        os.makedirs(la_dir)
    label_dst = "%s%s" % (os.path.join(la_dir,filename[:3]),".bmp")
    
    # 保存融合图像，存为BMP
    fusion_3c = np.stack((fusion_re,fusion_re,fusion_re)) # (C,H,W) 
    fusion_3c = fusion_3c.transpose(1,2,0) # (H,W,C) 224
    fusion_img = Image.fromarray(fusion_3c,"RGB")
    fusion_img.save(fusion_dst)
    
    # 保存label图像
    label_img = save_label(label_re)
    label_img.save(label_dst)
    
    # 保存CT图像
    ct_3c = np.stack((ct_re,ct_re,ct_re))
    ct_3c = ct_3c.transpose(1,2,0)
    ct_im = Image.fromarray(ct_3c,"RGB")
    ct_im.save(ct_dst)
    
    print(filename)

In [1]:
### 以下为测试一个病例

In [11]:
# 测试一个病人
patient_path = "F:\\data\\yuanfazhao\\yuanfazhao_data_05\\N0-60-wangguiping"

temp = os.listdir(patient_path)

all_ct_path =  os.path.join(patient_path,temp[0])
all_pet_path = os.path.join(patient_path,temp[3])
all_label_path = os.path.join(patient_path,temp[1])
print(all_ct_path)
print("\n")
print(all_pet_path)
print("\n")
print(all_label_path)
ct_path = [os.path.join(all_ct_path,f) for f in os.listdir(all_ct_path)]
print(ct_path)
pet_path = [os.path.join(all_pet_path,f) for f in os.listdir(all_pet_path)]
print(pet_path)
label_path = [os.path.join(all_label_path,f) for f in os.listdir(all_label_path)]
print(label_path)
for ctpath,petpath,labelpath in zip(ct_path,pet_path,label_path):
    # print(ctpath,petpath,labelpath)
    prep(ctpath,petpath,labelpath)

F:\data\yuanfazhao_data_05\N0-60-wangguiping\C5YGD35LZ


F:\data\yuanfazhao_data_05\N0-60-wangguiping\PFTBQFDSZ


F:\data\yuanfazhao_data_05\N0-60-wangguiping\Lable
['F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\C5YGD35LZ\\093.dcm', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\C5YGD35LZ\\094.dcm', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\C5YGD35LZ\\095.dcm']
['F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\PFTBQFDSZ\\093.dcm', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\PFTBQFDSZ\\094.dcm', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\PFTBQFDSZ\\095.dcm']
['F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\Lable\\093.bmp', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\Lable\\094.bmp', 'F:\\data\\yuanfazhao_data_05\\N0-60-wangguiping\\Lable\\095.bmp']
093.dcm
094.dcm
095.dcm


In [16]:
# 针对yuanfazhao_data_05处理数据
yuanfazhao_data = "F:\\data\\yuanfazhao\\yuanfazhao_data_05"
patient_path = [os.path.join(yuanfazhao_data,patient) for patient in os.listdir(yuanfazhao_data)]

ct_path = []
pet_path = []
label_path = []

for patient in patient_path:
    
    temp = os.listdir(patient)
    
    all_ct_path =  os.path.join(patient,temp[0])
    for f in os.listdir(all_ct_path):
        ct_path.append(os.path.join(all_ct_path,f))
        
    all_pet_path = os.path.join(patient,temp[3])
    for f in os.listdir(all_pet_path):
        pet_path.append(os.path.join(all_pet_path,f))
        
    all_label_path = os.path.join(patient,temp[1])
    for f in os.listdir(all_label_path):
        label_path.append(os.path.join(all_label_path,f))

# 执行操作
for ctpath,petpath,labelpath in zip(ct_path,pet_path,label_path):
    # print(ctpath,petpath,labelpath)
    prep(ctpath,petpath,labelpath)

087.dcm
088.dcm
089.dcm
090.dcm
106.dcm
107.dcm
108.dcm
109.dcm
110.dcm
111.dcm
112.dcm
088.dcm
089.dcm
090.dcm
091.dcm
092.dcm
093.dcm
094.dcm
095.dcm
096.dcm
090.dcm
091.dcm
092.dcm
093.dcm
094.dcm
095.dcm
096.dcm
097.dcm
098.dcm
099.dcm
100.dcm
098.dcm
099.dcm
100.dcm
101.dcm
102.dcm
103.dcm
104.dcm
105.dcm
106.dcm
107.dcm
108.dcm
109.dcm
093.dcm
094.dcm
095.dcm
084.dcm
085.dcm
086.dcm
087.dcm
088.dcm
089.dcm
090.dcm
091.dcm
092.dcm
093.dcm
094.dcm
095.dcm
096.dcm
097.dcm
127.dcm
128.dcm
129.dcm
130.dcm
131.dcm
132.dcm
133.dcm
134.dcm
135.dcm
136.dcm
137.dcm
118.dcm
119.dcm
120.dcm
121.dcm
122.dcm
123.dcm
124.dcm
125.dcm
126.dcm
127.dcm
132.dcm
133.dcm
134.dcm
135.dcm
136.dcm
137.dcm
138.dcm
139.dcm
140.dcm
141.dcm
142.dcm
143.dcm
103.dcm
104.dcm
105.dcm
106.dcm
107.dcm
108.dcm
109.dcm
110.dcm
111.dcm
112.dcm
113.dcm
114.dcm
115.dcm
116.dcm
117.dcm
118.dcm
119.dcm
120.dcm
121.dcm
122.dcm
090.dcm
091.dcm
092.dcm
093.dcm
094.dcm
095.dcm
096.dcm
097.dcm
098.dcm
099.dcm
100.dcm
101.dcm
